In [1]:
import pandas as pd

In [9]:
user_df=pd.read_table('https://static.turi.com/datasets/millionsong/10000.txt',header=None)

In [10]:
user_df.columns=['user_id','song_id','count']
print (user_df.head())

                                    user_id             song_id  count
0  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOAKIMP12A8C130995      1
1  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBBMDR12A8C13253B      2
2  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBXHDL12A81C204C0      1
3  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SOBYHAJ12A6701BF1D      1
4  b80344d063b5ccb3212f76538f3d9e43d87dca9e  SODACBL12A8C13C273      1


In [11]:
print (len(user_df['user_id'].unique()))

76353


In [6]:
song_info_df=pd.read_csv("https://static.turi.com/datasets/millionsong/song_data.csv")

In [7]:
print (song_info_df.head())

              song_id              title  \
0  SOQMMHC12AB0180CB8       Silent Night   
1  SOVFVAK12A8C1350D9        Tanssi vaan   
2  SOGTUKN12AB017F4F1  No One Could Ever   
3  SOBNYVR12A8C13558C      Si Vos Querés   
4  SOHSBXH12A8C13B0DF   Tangle Of Aspens   

                                release       artist_name  year  
0                 Monster Ballads X-Mas  Faster Pussy cat  2003  
1                           Karkuteillä  Karkkiautomaatti  1995  
2                                Butter    Hudson Mohawke  2006  
3                               De Culo       Yerba Brava  2003  
4  Rene Ablaze Presents Winter Sessions        Der Mystic     0  


In [ ]:
print ("hello")